In [1]:
import pathlib
import warnings

import bokeh
import pandas as pd
import geopandas as gpd
from bokeh.transform import factor_cmap
from bokeh.plotting import figure, output_file
from ipywidgets import interact, interact_manual
from bokeh.io import output_notebook, show, reset_output
from bokeh.models import ColumnDataSource, HoverTool, Legend, GeoJSONDataSource, LinearColorMapper, ColorBar

output_notebook()
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

Loading BokehJS ...

In [2]:
contiguous_usa = gpd.read_file('/data/us_shape/cb_2018_us_state_20m.shp')
contiguous_usa.head()

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
0,24,01714934,0400000US24,24,MD,Maryland,00,25151100280,6979966958,"MULTIPOLYGON (((-76.04621 38.02553, -76.00734 ..."
1,19,01779785,0400000US19,19,IA,Iowa,00,144661267977,1084180812,"POLYGON ((-96.62187 42.77925, -96.57794 42.827..."
2,10,01779781,0400000US10,10,DE,Delaware,00,5045925646,1399985648,"POLYGON ((-75.77379 39.72220, -75.75323 39.757..."
3,39,01085497,0400000US39,39,OH,Ohio,00,105828882568,10268850702,"MULTIPOLYGON (((-82.86334 41.69369, -82.82572 ..."
4,42,01779798,0400000US42,42,PA,Pennsylvania,00,115884442321,3394589990,"POLYGON ((-80.51989 40.90666, -80.51964 40.987..."


In [3]:
df = pd.DataFrame(columns=["state", "gender", "year", "name", "number"])
for path in pathlib.Path('/data').glob('*.TXT'):
    state = pd.read_csv(path, names=["state", "gender", "year", "name", "number"], header=None)
    df = df.append(state)

In [4]:
df.head()

,state,gender,year,name,number
0,IN,F,1910,Mary,619
1,IN,F,1910,Helen,324
2,IN,F,1910,Ruth,238
3,IN,F,1910,Dorothy,215
4,IN,F,1910,Mildred,200


## Gender trends over the years

In [5]:
gby_gyn = df[['gender', 'year', 'number']].groupby(['gender', 'year']).sum()
gby_gyn.reset_index(inplace=True)
gby_gyn.head()

,gender,year,number
0,F,1910,352089
1,F,1911,372381
2,F,1912,504298
3,F,1913,566973
4,F,1914,696906


In [6]:
# data
males = gby_gyn[gby_gyn["gender"] == "M"]
females = gby_gyn[gby_gyn["gender"] == "F"]

# plot 
hover = HoverTool()
hover.tooltips=[
    ('Year', '@year'),
    ('Count', '@number'),
    ('Gender', '@gender')
]

multi_line_plot = figure(plot_width=800, plot_height=500, toolbar_location='below')
source = ColumnDataSource(males)
multi_line_plot.line(x='year', y='number', source=source, color='blue', legend_label="Male", 
                     muted_color='blue', muted_alpha=0.2)
source = ColumnDataSource(females)
multi_line_plot.line(x='year', y='number', source=source, color='red', legend_label="Female",
                     muted_color='red', muted_alpha=0.2)
multi_line_plot.add_tools(hover)
multi_line_plot.legend.location = "top_left"
multi_line_plot.legend.click_policy="mute"

output_file("gender_trends.html", title="Gender Trends")
show(multi_line_plot)

In [7]:
males.rename(columns={"number":"male_count"}, inplace=True)
females.rename(columns={"number":"female_count"}, inplace=True)
male_female_df = pd.merge(males, females, how="outer", on=["year"])
male_female_df.head()

,gender_x,year,male_count,gender_y,female_count
0,M,1910,164226,F,352089
1,M,1911,193441,F,372381
2,M,1912,383703,F,504298
3,M,1913,461607,F,566973
4,M,1914,596440,F,696906


In [8]:
male_female_df["f_m_ratio"] = male_female_df["female_count"] / male_female_df["male_count"]

source = ColumnDataSource(male_female_df)
line_plot = figure(plot_width=500, plot_height=325, title='Female to Male ratio over the years', x_axis_label='years', y_axis_label='female to male ratio')
line_plot.line(x='year', y='f_m_ratio', line_width=2, source=source)

# add hover tool
hover = HoverTool()
hover.tooltips=[
    ('Year', '@year'),
    ('Ratio', '@f_m_ratio'),
    ('Male population', '@male_count'),
    ('Female population', '@female_count')
]
line_plot.add_tools(hover)
output_file("female_male_ratio.html", title="Female to Male ratio")
show(line_plot)

In [11]:
gender_pop_plot = figure(plot_width=600, plot_height=300)
gender_pop_plot.varea_stack(['female_count', 'male_count'],
                              x='year',
                              color=('pink', 'lightblue'),
                              legend_label=["females", "males"],
                              muted_color=('pink', 'lightblue'), 
                              muted_alpha=(0.2,0.2),
                              source=male_female_df)

gender_pop_plot.legend.location = "top_left"
gender_pop_plot.legend.click_policy="mute"
output_file("gender_population_trends.html", title="Population Trends")
show(gender_pop_plot)

## Most populated state trends

In [12]:
gby_sy = df[['state', 'year', 'number']].groupby(['state', 'year']).sum()
gby_sy.reset_index(inplace=True)
gby_sy.head()

,state,year,number
0,AK,1910,115
1,AK,1911,84
2,AK,1912,141
3,AK,1913,110
4,AK,1914,245


In [13]:
year_pop = gby_sy[["year", "number"]].groupby("year").sum().rename(columns={"number":"total_population"})
gby_sy = gby_sy.join(year_pop, on="year")
gby_sy["pop_percent"] = (gby_sy["number"]/gby_sy["total_population"])*100
gby_sy.head()

,state,year,number,total_population,pop_percent
0,AK,1910,115,516315,0.022273
1,AK,1911,84,565822,0.014846
2,AK,1912,141,888001,0.015878
3,AK,1913,110,1028580,0.010694
4,AK,1914,245,1293346,0.018943


In [14]:
pop = gby_sy.groupby(['year']).apply(lambda x: x['pop_percent'].nlargest(5)).reset_index()
pop.drop(columns=["level_1"], inplace=True)
pop = pd.merge(pop, gby_sy,  how='inner', on=["year", "pop_percent"])
pop.head()

,year,pop_percent,state,number,total_population
0,1910,9.325315,NY,48148,516315
1,1910,8.757057,PA,45214,516315
2,1910,5.498581,TX,28390,516315
3,1910,5.161771,IL,26651,516315
4,1910,4.762403,GA,24589,516315


In [16]:
pop_top5 = pop.groupby("year")["pop_percent", "state"].apply(lambda x: zip(x["pop_percent"].tolist(), x["state"].tolist()))

pop_top5 = pd.DataFrame(pop_top5.tolist(), index=v.index)\
       .rename(columns=lambda x: x + 1)\
       .add_prefix('pop_percent')\
       .reset_index()
pop_top5.head()

pop_top5['pop_percent1'], pop_top5['state1'] = pop_top5.pop_percent1.str
pop_top5['pop_percent2'], pop_top5['state2'] = pop_top5.pop_percent2.str
pop_top5['pop_percent3'], pop_top5['state3'] = pop_top5.pop_percent3.str
pop_top5['pop_percent4'], pop_top5['state4'] = pop_top5.pop_percent4.str
pop_top5['pop_percent5'], pop_top5['state5'] = pop_top5.pop_percent5.str
pop_top5.head()

,year,pop_percent1,pop_percent2,pop_percent3,pop_percent4,pop_percent5,state1,state2,state3,state4,state5
0,1910,9.325315,8.757057,5.498581,5.161771,4.762403,NY,PA,TX,IL,GA
1,1911,10.745606,9.293382,5.068378,5.034622,4.346420,NY,PA,TX,IL,OH
2,1912,10.365304,9.137264,5.700106,5.085129,4.611369,NY,PA,IL,TX,OH
3,1913,10.046180,9.133174,5.967256,5.241887,4.801571,NY,PA,IL,TX,OH
4,1914,10.580231,9.426403,5.744171,4.988379,4.740495,NY,PA,IL,TX,OH


In [17]:
from bokeh.palettes import magma

In [18]:
color_map={"NY":"#FEC78B", "CA":"#472C7B", "TX":"#E75262", "IL": "#2B738E", 
           "OH":"#842681", "GA": "#50127B", "PA": "#1C1046", "FL":"#000003"}

pop_top5["color1"] =  pop_top5["state1"].apply(lambda x: color_map[x])
pop_top5["color2"] =  pop_top5["state2"].apply(lambda x: color_map[x])
pop_top5["color3"] =  pop_top5["state3"].apply(lambda x: color_map[x])
pop_top5["color4"] =  pop_top5["state4"].apply(lambda x: color_map[x])
pop_top5["color5"] =  pop_top5["state5"].apply(lambda x: color_map[x])
pop_top5.head()

,year,pop_percent1,pop_percent2,pop_percent3,pop_percent4,pop_percent5,state1,state2,state3,state4,state5,color1,color2,color3,color4,color5
0,1910,9.325315,8.757057,5.498581,5.161771,4.762403,NY,PA,TX,IL,GA,#FEC78B,#1C1046,#E75262,#2B738E,#50127B
1,1911,10.745606,9.293382,5.068378,5.034622,4.346420,NY,PA,TX,IL,OH,#FEC78B,#1C1046,#E75262,#2B738E,#842681
2,1912,10.365304,9.137264,5.700106,5.085129,4.611369,NY,PA,IL,TX,OH,#FEC78B,#1C1046,#2B738E,#E75262,#842681
3,1913,10.046180,9.133174,5.967256,5.241887,4.801571,NY,PA,IL,TX,OH,#FEC78B,#1C1046,#2B738E,#E75262,#842681
4,1914,10.580231,9.426403,5.744171,4.988379,4.740495,NY,PA,IL,TX,OH,#FEC78B,#1C1046,#2B738E,#E75262,#842681


In [19]:
hover = HoverTool()
hover.tooltips=[
    ('Year', '@year'),
    ('State1', "(" + '@state1' + " " + '@pop_percent1' + ")"),
    ('State2', "(" + '@state2' + " " + '@pop_percent2' + ")"),
    ('State3', "(" + '@state3' + " " + '@pop_percent3' + ")"),
    ('State4', "(" + '@state4' + " " + '@pop_percent4' + ")"),
    ('State5', "(" + '@state5' + " " + '@pop_percent5' + ")")
]

pop_top5_df = ColumnDataSource(pop_top5)

pop_top5_chart = figure(plot_width=1000, plot_height=500, title='Top 5 populated states', toolbar_location='below', 
                           x_axis_label='year', y_axis_label='percepent of total population')

pop_top5_chart.vbar_stack(['pop_percent5', 'pop_percent4', 'pop_percent3', 'pop_percent2', 'pop_percent1'], 
                             x='year', 
                             width=0.8, 
                             color=['color5', 'color4', 'color3', 'color2', 'color1'],
                             legend_group=['state5', 'state4', 'state3', 'state2', 'state1'],
                             source=pop_top5_df)

pop_top5_chart.add_tools(hover)
for p in pop_top5_chart.legend:
    legend_tmp = {x.label['value']: x for x in p.items}
    p.items.clear()
    p.items.extend(legend_tmp.values())
    
pop_top5_chart.legend.location = "top_center"
pop_top5_chart.legend.orientation = "horizontal"

output_file("population_trends.html", title="Gender Trends")

show(pop_top5_chart)

In [20]:
gby_sy.head()

,state,year,number,total_population,pop_percent
0,AK,1910,115,516315,0.022273
1,AK,1911,84,565822,0.014846
2,AK,1912,141,888001,0.015878
3,AK,1913,110,1028580,0.010694
4,AK,1914,245,1293346,0.018943


In [23]:
pop_states = contiguous_usa.merge(gby_sy, left_on = "STUSPS", right_on = "state")
# Drop Alaska and Hawaii
pop_states = pop_states.loc[~pop_states['NAME'].isin(['Alaska', 'Hawaii'])]

In [42]:
from bokeh.palettes import cividis
@interact_manual
def visulaize_us_population(year=range(1910, 2020, 1)):
    
    map_states = pop_states[pop_states["year"] == year]
    geosource = GeoJSONDataSource(geojson = map_states.to_json())
    p = figure(title = 'Population density of {}'.format(year), 
               plot_height = 600 ,
               plot_width = 950, 
               toolbar_location = 'below',
               tools = "pan, wheel_zoom, box_zoom, reset")
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    high = max(map_states.pop_percent)
    color_mapper = LinearColorMapper(palette = cividis(16), low = 0, high = high)
    # Add patch renderer to figure.
    states = p.patches('xs','ys', source = geosource,
                       fill_color = {'field' :'pop_percent',
                                     'transform' : color_mapper},
                       line_color = "gray", 
                       line_width = 0.25, 
                       fill_alpha = 1)
    # Create hover tool
    p.add_tools(HoverTool(renderers = [states],
                          tooltips = [('State','@NAME'), ('Population','@pop_percent'), ('Year', '@year')]))

    color_bar = ColorBar(color_mapper = color_mapper, 
                         label_standoff = 8,
                         width = 500, height = 20,
                         border_line_color = None,
                         location = (0,0), 
                         orientation = 'horizontal',
                         major_label_overrides = {'0': '0', str(high):str(high)})
    p.add_layout(color_bar, 'below')

    output_file("population_trends_in_{}.html".format(year), title="Gender Trends")
    show(p)

interactive(children=(Dropdown(description='year', options=(1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 19…

## Popular names

In [25]:
df = df.astype({'number':'int64'})

In [26]:
popular_names = df.groupby(['year', "state"])["number", "name"].apply(lambda x: x.nlargest(5, columns=['number'])).reset_index()

v = popular_names.groupby(["year", "state"])["number", "name"].apply(lambda x: zip(x["number"].tolist(), x["name"].tolist()))
    
names_frequency = pd.DataFrame(v.tolist(), index=v.index)\
           .rename(columns=lambda x: x + 1)\
           .add_prefix('frequent')\
           .reset_index()
for i in range(1, 6):
    names_frequency['frequent' + str(i)], names_frequency['name' + str(i)] = names_frequency['frequent' + str(i)].str
    
names_frequency.head()


,year,state,frequent1,frequent2,frequent3,frequent4,frequent5,name1,name2,name3,name4,name5
0,1910,AK,14,12,10,8,8,Mary,Annie,Anna,Margaret,John
1,1910,AL,875,486,482,399,343,Mary,James,Annie,John,William
2,1910,AR,408,231,203,188,148,Mary,James,William,John,Ruby
3,1910,AZ,74,39,29,29,27,Mary,John,Maria,Manuel,Alice
4,1910,CA,295,239,237,220,170,Mary,Helen,John,Dorothy,William


## Popular names by gender

In [27]:
popular_names_by_gender = df.groupby(['year', "state", "gender"])["number", "name"].apply(lambda x: x.nlargest(5, columns=['number'])).reset_index()

v = popular_names_by_gender.groupby(["year", "state", "gender"])["number", "name"].apply(lambda x: zip(x["number"].tolist(), x["name"].tolist()))
    
names_frequency_by_gender = pd.DataFrame(v.tolist(), index=v.index)\
           .rename(columns=lambda x: x + 1)\
           .add_prefix('frequent')\
           .reset_index()
for i in range(1, 6):
    names_frequency_by_gender['frequent' + str(i)], names_frequency_by_gender['name' + str(i)] = names_frequency_by_gender['frequent' + str(i)].str
    
names_frequency_by_gender.head()

,year,state,gender,frequent1,frequent2,frequent3,frequent4,frequent5,name1,name2,name3,name4,name5
0,1910,AK,F,14,12,10,8,7.0,Mary,Annie,Anna,Margaret,Helen
1,1910,AK,M,8,7,6,6,5.0,John,James,Paul,Robert,Carl
2,1910,AL,F,875,482,257,232,204.0,Mary,Annie,Willie,Mattie,Ruby
3,1910,AL,M,486,399,343,343,277.0,James,John,William,Willie,Robert
4,1910,AR,F,408,148,140,132,109.0,Mary,Ruby,Ruth,Willie,Ethel


In [43]:
@interact_manual
def box_plot(year=['any'] + list(df.year.unique()), state=['any'] + list(df.state.unique()), data_type=["all", "M", "F"]):
    colors = ['#000003', '#410967', '#932567', '#DC5039', '#FBA40A']
    if data_type == "all":
        frequency = names_frequency
        popular = popular_names
    else:
        frequency = names_frequency_by_gender[names_frequency_by_gender["gender"]==data_type]
        popular = popular_names_by_gender[popular_names_by_gender["gender"] == data_type]
        
    if 'any' in [state, year]:
        assert (state == 'any' and year == 'any') == False
        if state == 'any':
            tmp1 = frequency[(frequency["year"]==year)]
            for i in range(1, 6):
                tmp1["color" + str(i)] = colors[i-1]
            tmp1.drop(columns=['year'], inplace=True)
            hover = HoverTool()
            hover.tooltips=[
                ('State', '@state'),
                ('Name1', '@name1'),
                ('Name2', '@name2'),
                ('Name3', '@name3'),
                ('Name4', '@name4'),
                ('Name5', '@name5'),
            ]
            title = 'Most popular names in year {} across all states'.format(year)
            x = 'state'
            stacked_bar_chart = figure(x_range=tmp1[x].unique(), plot_width=1000, plot_height=500, 
                                   title=title, toolbar_location='below')
        else:
            tmp1 = frequency[(frequency["state"]==state)]
            for i in range(1, 6):
                tmp1["color" + str(i)] = colors[i-1]
            tmp1.drop(columns=['state'], inplace=True)
            hover = HoverTool()
            hover.tooltips=[
                ('Year', '@year'),
                ('Name1', '@name1'),
                ('Name2', '@name2'),
                ('Name3', '@name3'),
                ('Name4', '@name4'),
                ('Name5', '@name5'),
            ]
            title = 'Most popular names in state {} over the years'.format(state)
            x = 'year'
            stacked_bar_chart = figure(plot_width=1000, plot_height=500, 
                                   title=title, toolbar_location='below')

        
        cds_stacked_bar_df = ColumnDataSource(tmp1)
        
        stacked_bar_chart.vbar_stack(['frequent1', 'frequent2', 'frequent3', 'frequent4', 'frequent5'], 
                                     x=x, 
                                     width=0.8, 
                                     color=['color1', 'color2', 'color3', 'color4', 'color5'],
                                     source=cds_stacked_bar_df)

        stacked_bar_chart.add_tools(hover)
        output_file("population_names.html", title="Popular names")
        show(stacked_bar_chart)
    else:
        tmp1 = popular[(popular["year"] == year) & (popular["state"] == state)]
        hover = HoverTool()
        hover.tooltips=[
            ('Name', '@name'),
        ]
        cds_stacked_bar_df = ColumnDataSource(tmp1)
        p = figure(x_range=tmp1.name, plot_height=250, 
                   title="Most popular names in state {}, in the year {}".format(state, year),
                   toolbar_location=None, tools="")
        p.vbar('name', top='number', width=0.9, source=cds_stacked_bar_df, line_color='white', 
               fill_color=factor_cmap('name', palette=colors, factors=tmp1.name))
        p.add_tools(hover)
        output_file("population_names_in_{}_{}.html".format(year, state), title="Popular names")
        show(p)
        


interactive(children=(Dropdown(description='year', options=('any', 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1…

In [29]:
box_plot(2012, "IN", "F")

## Gender Neutrality

In [30]:
neutral = df.groupby(["year", "name"]).agg({"number":sum, "gender":set})
neutral = neutral[neutral["gender"] == {"M", "F"}]
neutral.head()

number  gender
year name                   
1910 Alva         58  {M, F}
     Annie      3498  {M, F}
     Bennie      223  {M, F}
     Beverly      17  {M, F}
     Billie      131  {M, F}

In [34]:
neutral_by_year = pd.DataFrame(neutral.reset_index().groupby(["year"]).apply(lambda x: x.nlargest(5, columns=['number'])))
neutral_by_year = neutral_by_year[["name", "number"]].reset_index()
neutral_by_year.head(10)

,year,level_1,name,number
0,1910,57,Mary,22907
1,1910,39,John,11479
2,1910,27,Helen,10484
3,1910,32,James,9203
4,1910,80,William,8849
5,1911,140,Mary,24450
6,1911,127,John,13469
7,1911,113,Helen,11811
8,1911,160,William,10603
9,1911,119,James,9980


In [35]:
from bokeh.palettes import Spectral5
@interact_manual
def vbar_plot(year=list(df.year.unique())):
    neutral_y = neutral_by_year[(neutral_by_year["year"] == year)]
    neutral_y_df = ColumnDataSource(neutral_y)
    p = figure(x_range=neutral_y.name, plot_height=250, 
               title="Most gender neutral names in the year {}".format(year),
               toolbar_location=None, tools="")
    p.vbar('name', top='number', width=0.9, source=neutral_y_df, line_color='white', 
    fill_color=factor_cmap('name', palette=Spectral5, factors=neutral_y.name))
    show(p)

interactive(children=(Dropdown(description='year', options=(1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 19…

In [36]:
def gender_neutrality(t):
    males = t[t["gender"] == "M"]
    females = t[t["gender"] == "F"]
    n = len(set(males).intersection(set(females)))
    return n*100/(len(males)+len(females))
neutrality = pd.DataFrame(df.groupby(["year", "state"])
                          .apply(lambda t: gender_neutrality(t)), 
                          columns=["neutrality"]).reset_index()
neutrality.head()

,year,state,neutrality
0,1910,AK,31.250000
1,1910,AL,0.813008
2,1910,AR,1.035197
3,1910,AZ,4.854369
4,1910,CA,1.377410


In [37]:
from bokeh.palettes import inferno
# plot 
hover = HoverTool()
hover.tooltips=[
    ('Year', '@year'),
    ('Neutrality percentage', '@neutrality'),
    ('State', '@state')
]

multi_line_plot = figure(plot_width=800, plot_height=500, toolbar_location='below')
legend = []
for state, color in zip(neutrality.state.unique(), inferno(51)):
    source = ColumnDataSource(neutrality[neutrality["state"]==state])
    r = multi_line_plot.line(x='year', y='neutrality', source=source, color=color, 
                             muted_color=color, muted_alpha=0.2)
    legend += [(state, [r])]

multi_line_plot.add_tools(hover)

legend = Legend(items=legend, location="center", click_policy="mute", label_text_font_size = "8px")

multi_line_plot.add_layout(legend, 'right')
show(multi_line_plot)

## Correlation between names of different states

In [38]:
state1 = df[(df['state'] == "IN") & (df["year"] == 1910)][["name", "gender", "number"]]
state2 = df[(df["state"] == "CA") & (df["year"] == 1910)][["name", "gender", "number"]]
state1['name']=state1['name'].astype('category').cat.codes
state2['name']=state2['name'].astype('category').cat.codes
state1['gender']= state1['gender'].astype('category').cat.codes
state2['gender']= state2['gender'].astype('category').cat.codes
print(state1.head(), state2.head())

   name  gender  number
0   258       0     619
1   153       0     324
2   318       0     238
3    75       0     215
4   269       0     200    name  gender  number
0   255       0     295
1   156       0     239
2    87       0     220
3   242       0     163
4   126       0     134


In [39]:
state1.corrwith(state2, axis = 0)

name     -0.091594
gender         NaN
number    0.961985
dtype: float64